# Fill out missing chips

The initial extended evoland chips were gotten from AWS Element84 S2 COGs. This dataset is missing a bunch of recently reprocessed sentinel 2 data from the beginning of the data availability.

This notebook aims to fill in any gaps using data from the official CDSE repository. 

To do this we follow a few steps. 

1. Get all dates of the chips available already
2. Cross-reference with CDSE stac catalog
3. Use SH Processing API to query missing chips

To find the best way to do this we need to find out approximate numbers of how many chips are missing.

In [1]:
from pathlib import Path
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import traceback

from tqdm import tqdm
import rasterio
import geopandas as gpd
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    SHConfig,
    SentinelHubCatalog,
)

import numpy as np

c:\Users\Jonas.Viehweger\Documents\Projects\2025\DISFOR\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
samples = gpd.read_parquet("../data/samples.parquet")

In [3]:
samples

,sample_id,original_sample_id,interpreter,dataset,source,source_description,s2_tile,cluster_id,cluster_description,comment,confidence,geometry
0,0,0,vij,Evoland,EFFIS,"Evoland Project, EFFIS Source of Wildfire Poly...",30SUF,0.0,Damage polygons,"border, thinning, then clear cut",high,POINT (-4.12212 36.74179)
1,1,1,vij,Evoland,EFFIS,"Evoland Project, EFFIS Source of Wildfire Poly...",30SUF,1.0,Damage polygons,"unclear progression, edge",medium,POINT (-4.12161 36.74231)
2,2,2,vij,Evoland,EFFIS,"Evoland Project, EFFIS Source of Wildfire Poly...",30SUF,2.0,Damage polygons,plantation,high,POINT (-4.1192 36.74203)
3,3,3,vij,Evoland,EFFIS,"Evoland Project, EFFIS Source of Wildfire Poly...",30SUF,3.0,Damage polygons,plantation,high,POINT (-4.12845 36.75831)
4,4,4,vij,Evoland,EFFIS,"Evoland Project, EFFIS Source of Wildfire Poly...",30SUF,5.0,Damage polygons,clear cut,high,POINT (-4.12816 36.75908)
...,...,...,...,...,...,...,...,...,...,...,...,...
3818,3818,16066,vij,Windthrow,FORWIND + Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,<NA>,SI20200205,"Id of the Event, given as ISO2 + Date of storm",<NA>,high,POINT (14.39175 46.2299)
3819,3819,16067,vij,Windthrow,FORWIND + Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,<NA>,SI20200205,"Id of the Event, given as ISO2 + Date of storm",unclear salvage,high,POINT (14.4004 46.2291)
3820,3820,16070,vij,Windthrow,FORWIND + Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,<NA>,SI20200205,"Id of the Event, given as ISO2 + Date of storm",<NA>,high,POINT (14.39046 46.24575)
3821,3821,16071,vij,Windthrow,FORWIND + Copernicus Emergency Service,https://mapping.emergency.copernicus.eu/activa...,<NA>,SI20200205,"Id of the Event, given as ISO2 + Date of storm",unclear salvage,high,POINT (14.39247 46.2441)


In [4]:
config = SHConfig("default-profile")

In [5]:
evalscript_body = """
AVAILABLE = new Set();
NOT_AVAILABLE = new Set();
TO_INCLUDE_SET = new Set(TO_INCLUDE);

function setup() {
    return {
        input: [{
            bands: ["B02","B03","B04","B05","B06","B07","B08","B8A","B11","B12","SCL"],
            units: "DN"
        }],
        output: TO_INCLUDE.map(date => ({
            id: date,
            bands: 11,
            sampleType: "INT16"
        })),
        mosaicking: "ORBIT"
    };
}

function preProcessScenes(collections) {
    collections.scenes.orbits = collections.scenes.orbits.filter(function(orbit) {
        var orbitDateFrom = orbit.dateFrom.split("T")[0];
        var toGet = TO_INCLUDE_SET.has(orbitDateFrom);
        if(toGet){
          AVAILABLE.add(orbitDateFrom)
        }
        return toGet
    });
    return collections;
}

function updateOutputMetadata(scenes, inputMetadata, outputMetadata){
  outputMetadata.userData = Array.from(AVAILABLE)
}

function evaluatePixel(samples) {
    let dataOutputs = {};
    var i = 0;
    for(let key of TO_INCLUDE){
      if(AVAILABLE.has(key)){
        let sample = samples[i];
        dataOutputs[key] = [
            sample.B02,
            sample.B03,
            sample.B04,
            sample.B05,
            sample.B06,
            sample.B07,
            sample.B08,
            sample.B8A,
            sample.B11,
            sample.B12,
            sample.SCL
        ];
        i++;
      } else {
        dataOutputs[key] = [0]
      }
    }
    return dataOutputs;
}
"""

In [7]:
def process_sample(sample, evalscript_body, config):
    """Process a single sample - designed to be run in parallel."""
    sample_paths = list(Path(f"../data/tiffs/{sample.sample_id}").glob("*.tif"))
    existing_timestamps = {path.stem for path in sample_paths}
    centroid = sample.geometry
    lat, lon = centroid.y, centroid.x

    bbox = BBox((lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5), crs=CRS.WGS84)
    time_interval = "2015-01-01", "2025-01-01"

    catalog = SentinelHubCatalog(config=config)
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=bbox,
        time=time_interval,
        filter="eo:cloud_cover < 80",
        fields={
            "include": ["id", "properties.datetime", "properties.eo:cloud_cover"],
            "exclude": [],
        },
    )

    results = list(search_iterator)
    available_timestamps = {item["properties"]["datetime"][0:10] for item in results}
    missing_timestamps = list(available_timestamps - existing_timestamps)

    if not missing_timestamps:
        return sample.sample_id, 0, None

    missing_timestamps.sort()

    # Open first image and get bbox and crs
    with rasterio.open(sample_paths[0]) as src:
        profile = src.profile
        epsg = src.crs
        bbox = src.bounds

    sh_bbox = BBox(bbox=list(bbox), crs=CRS(epsg.to_epsg()))
    evalscript = (
        "//VERSION 3\n"
        + "TO_INCLUDE = "
        + json.dumps(missing_timestamps)
        + evalscript_body
    )

    request_all_bands = SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A.define_from(
                    "s2l2a", service_url=config.sh_base_url
                ),
                time_interval=(missing_timestamps[0], missing_timestamps[-1]),
                mosaicking_order=MosaickingOrder.LEAST_RECENT,
            )
        ],
        responses=[
            SentinelHubRequest.output_response(output, MimeType.TIFF)
            for output in missing_timestamps
        ]
        + [SentinelHubRequest.output_response("userdata", MimeType.JSON)],
        bbox=sh_bbox,
        size=(32, 32),
        config=config,
    )

    all_bands_response = request_all_bands.get_data()
    to_save = list(all_bands_response[-1]["userdata.json"].values())
    out_path = Path(f"../data/tiffs/{sample.sample_id}")

    for date in to_save:
        filename = f"{date}.tif"
        with rasterio.open(out_path / filename, "w", **profile) as dst:
            dst.write(np.transpose(all_bands_response[-1][filename], (2, 0, 1)))

    return sample.sample_id, len(to_save), None

In [8]:
# Main execution with ThreadPoolExecutor
max_workers = 10  # Adjust based on your system and API rate limits

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    futures = {
        executor.submit(
            process_sample, sample, evalscript_body, config
        ): sample.sample_id
        for _, sample in samples.query("dataset == 'Windthrow'").iterrows()
    }

    # Process completed tasks as they finish
    for future in tqdm(as_completed(futures), total=len(futures)):
        sample_id = futures[future]
        try:
            result_id, message, error = future.result()
            print(f"Sample {result_id}: {message}")
        except Exception as e:
            print(f"Sample {sample_id} generated an exception: {e}")
            traceback.print_exc()

  0%|          | 1/294 [00:03<18:03,  3.70s/it]

Sample 3530: 0


  2%|▏         | 5/294 [00:04<02:35,  1.86it/s]

Sample 3538: 0
Sample 3529: 0
Sample 3535: 0
Sample 3534: 0


  2%|▏         | 7/294 [00:04<01:50,  2.61it/s]

...

Sample 3780 generated an exception: Failed to download from:
https://sh.dataspace.copernicus.eu/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://sh.dataspace.copernicus.eu/api/v1/process
Server response: "{"status": 400, "reason": "Bad Request", "message": "Failed to evaluate script!\nevalscript.js:45: TypeError: Cannot read property 'B02' of undefined\n            sample.B02,\n                  ^\nTypeError: Cannot read property 'B02' of undefined\n    at evaluatePixel (evalscript.js:45:19)\n    at executeForMultipleScenes (<anonymous>:1286:14)\n", "code": "COMMON_EXCEPTION"}"
Sample 3773 generated an exception: Failed to download from:
https://sh.dataspace.copernicus.eu/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://sh.dataspace.copernicus.eu/api/v1/process
Server response: "{"status": 400, "reason": "Bad Request", "message": "Failed to evaluate script!\nevalscript.js:45: TypeError: Cannot read property 'B02' of undefined\n 

Traceback (most recent call last):
  File "c:\Users\Jonas.Viehweger\Documents\Projects\2025\DISFOR\.venv\Lib\site-packages\sentinelhub\download\handlers.py", line 40, in new_download_func
    return download_func(self, request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jonas.Viehweger\Documents\Projects\2025\DISFOR\.venv\Lib\site-packages\sentinelhub\download\sentinelhub_client.py", line 100, in _execute_download
    response.raise_for_status()
  File "c:\Users\Jonas.Viehweger\Documents\Projects\2025\DISFOR\.venv\Lib\site-packages\requests\models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://sh.dataspace.copernicus.eu/api/v1/process

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Jonas.Viehweger\AppData\Local\Temp\ipykernel_21128\853727372.py", line 17, in <module>
    result_id, 

Sample 3714 generated an exception: Failed to download from:
https://sh.dataspace.copernicus.eu/api/v1/process
with ReadTimeout:
HTTPSConnectionPool(host='sh.dataspace.copernicus.eu', port=443): Read timed out. (read timeout=120.0)
There might be a problem in connection or the server failed to process your request. Please try again.


Traceback (most recent call last):
  File "c:\Users\Jonas.Viehweger\Documents\Projects\2025\DISFOR\.venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jonas.Viehweger\Documents\Projects\2025\DISFOR\.venv\Lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Jonas.Viehweger\AppData\Roaming\uv\python\cpython-3.12.10-windows-x86_64-none\Lib\http\client.py", line 1430, in getresponse
    response.begin()
  File "C:\Users\Jonas.Viehweger\AppData\Roaming\uv\python\cpython-3.12.10-windows-x86_64-none\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Jonas.Viehweger\AppData\Roaming\uv\python\cpython-3.12.10-windows-x86_64-none\Lib\http\client.py", line 292, in 

Sample 3790 generated an exception: Failed to download from:
https://sh.dataspace.copernicus.eu/api/v1/process
with ReadTimeout:
HTTPSConnectionPool(host='sh.dataspace.copernicus.eu', port=443): Read timed out. (read timeout=120.0)
There might be a problem in connection or the server failed to process your request. Please try again.


In [9]:
futures

{<Future at 0x22c414e5ee0 state=finished returned tuple>: 3529,
 <Future at 0x22c414e5550 state=finished returned tuple>: 3530,
 <Future at 0x22c414e51f0 state=finished returned tuple>: 3531,
 <Future at 0x22c414e6510 state=finished returned tuple>: 3532,
 <Future at 0x22c414e67e0 state=finished returned tuple>: 3533,
 <Future at 0x22c414e6ea0 state=finished returned tuple>: 3534,
 <Future at 0x22c414e7320 state=finished returned tuple>: 3535,
 <Future at 0x22c414e76e0 state=finished raised DownloadFailedException>: 3536,
 <Future at 0x22c414e7b90 state=finished returned tuple>: 3537,
 <Future at 0x22c414e7ef0 state=finished returned tuple>: 3538,
 <Future at 0x22c41594110 state=finished returned tuple>: 3539,
 <Future at 0x22c41594470 state=finished returned tuple>: 3540,
 <Future at 0x22c415945f0 state=finished returned tuple>: 3541,
 <Future at 0x22c41594770 state=finished returned tuple>: 3542,
 <Future at 0x22c415948f0 state=finished returned tuple>: 3543,
 <Future at 0x22c41594aa